In [32]:
import scipy as sc
import numpy as np
#!pip install gpxpy
import gpxpy
from geopy import distance
import json
import pandas as pd
import math
!pip install fit2gpx
from fit2gpx import Converter


In [118]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


# Putting it all together:

1. read gpx file(s) into an array with elevations and coordinates
2. read segement data into array with elevations and coordinates
3. get total distance and total elevation difference from segment
4. sweep through gpx arrays to find sections that roughly fit the segment
5. calculate elapsed times for those sections
6. use DTW to see if segments are actually similar
    - if they are, compare times.

- Thought - skip the prelim data sweep and just use DTW

In [96]:
def dtw_window(s, t, window):
    lenS, lenT = len(s), len(t)
    w = np.max([window, abs(lenS-lenT)])
    dtw_matrix = np.zeros((lenS+1, lenT+1))
    
    for i in range(lenS+1):
        for j in range(lenT+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, lenS+1):
        for j in range(np.max([1, i-w]), np.min([lenT, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, lenS+1):
        for j in range(np.max([1, i-w]), np.min([lenT, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            #print(cost)
            prev_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + prev_min
    return dtw_matrix

In [34]:
def dtw(s, t):
    lenS, lenT = len(s), len(t)
    dtw_matrix = np.zeros((lenS+1, lenT+1))
    for i in range(lenS+1):
        for j in range(lenT+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, lenS+1):
        for j in range(1, lenT+1):
            cost = abs(s[i-1] - t[j-1]) # cost from box
            prev_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]]) # get last min
            dtw_matrix[i, j] = cost + prev_min # calculate cost/distance - last element is total distance
    return dtw_matrix

In [35]:
from geopy import distance
import math
# method to calculate distances between point
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).m
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance

In [36]:
from geopy import Point
import math
# method to make GPX file into array of elevations, times, distances
def file_to_elevation_arr(fileName):
        gpx_file = open(fileName, 'r')
        gpx = gpxpy.parse(gpx_file)
        elevevation_time_and_distance = []
        pointOne = None
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    pointTwo = point
                    distance = 0
                    if pointOne:
                        p1 = Point([pointOne.latitude, pointOne.longitude, pointOne.elevation])
                        p2 = Point([pointTwo.latitude, pointTwo.longitude, pointTwo.elevation])
                        distance = distanceBetweenPoints(p1, p2)
                    pointOne = point
                    #print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))
                    elevevation_time_and_distance.append((point.elevation,point.time, distance))
        return elevevation_time_and_distance

In [39]:
#segment = "data/Elevation JSON/segment_location_data_elevation_0.json"

In [40]:
# takes a panda dataframe and converts to list of tuples
def dataframe_segment_to_list(df):
    sg =[]
    # Iterate over each row
    for index, rows in df.iterrows():
        # Create list for the current row
        my_list =[rows[2], rows[3]]
        #print(my_list)
        
        # append the list to the final list
        sg.append(my_list)
    return sg
  

In [41]:
#iterates through segment to give total elevation gain/loss
def total_segment_elevation(segment_data):
    total_elev = segment_data[len(segment_data)-1][0] - segment_data[0][0]
    return total_elev

#iterates through segment to give total distance gain
def total_segment_distance(segment_data):
    total_distance = 0
    for i in segment_data:
        total_distance += i[1].item()
    return total_distance

In [42]:
# old method that is slow - sweeps over every single point
def sweep_user_activity_old(user_data, segement_data, x):
    list_of_matches = [] # list of lists

    seg_distance = total_segment_distance(segement_data)

    #print(seg_distance)
    seg_elevation = total_segment_elevation(segement_data)
    #print(seg_elevation)
    
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        # until we get to the window 
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window += 1
    delta_distance = user_distance - user_data[0][2]
    # now we have the window, so we need to iterate through and see if the profiles match
    second_window = 0
    second_distance = 0
    user_elev_two = user_data[0][0]
    while user_distance < user_data[-1][2] and first_window < len(user_data):
        user_elev_two = user_data[first_window][0]
        user_elev = user_data[second_window][0]
        if user_elev_two - user_elev > seg_elevation-5 and  user_elev_two - user_elev < seg_elevation+5:
            list_of_matches.append(user_data[second_window:first_window])
            #print(first_window, second_window)
            # add match to list
        user_distance += user_data[first_window][2]
        second_distance += user_data[second_window][2]
        first_window +=1
        second_window +=1
    return list_of_matches




In [95]:

# Go to segment distance, when the segment distance matches the user window, compare elevations
# this method only compares elevations every 100 GPS points
def sweep_user_activity(user_data, segement_data, window):
    list_of_matches = [] # list of lists
    seg_distance = total_segment_distance(segement_data) # total segment distance
    seg_elevation = total_segment_elevation(segement_data) # total segment elevation
    
    # go to the correct distance in user
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window+=1
    second_window = 0 
    second_distance = 0
    while first_window < len(user_data)-1: 
        while user_distance-second_distance < seg_distance and first_window < len(user_data)-1:
            user_distance += user_data[first_window][2]
            first_window+=1
        user_elev_two = user_data[first_window][0]
        user_elev = user_data[second_window][0]
        second_distance += user_data[second_window][2]
        if first_window % 100 == 0:
            if user_elev_two - user_elev > seg_elevation-window and  user_elev_two - user_elev < seg_elevation+window:
                list_of_matches.append(user_data[second_window:first_window])
        second_window+= 1
    return list_of_matches
        


In [45]:
# this method reads a segment JSON (must be prepped by detailed_segment_data_maps.ipynb and coordinates_to_elevation.ipynb)
def read_segment(segmentFile):
    segment = segmentFile
    df = pd.read_json(segment)
    df.head()
    seg = dataframe_segment_to_list(df)
    return seg

In [46]:
# this could be any segment, just for testing
claremont_turkey_trot = read_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/james_segment_location_data_elevation_0.json')


In [49]:
import numpy as np
import matplotlib.pyplot as plt
def graphing(user, segment):
    total_user_distance = 0
    total_segment_distance1 = 0
    user_start_elev = user[0][0]
    segment_start_elev = segment[0][0]
    user_dtw_list = []
    segment_dtw_list = []
    user_dtw_distance = []
    seg_dtw_distance = []
    for i in range(len(user)):
        total_user_distance += user[i][2]
        user_dtw_distance.append(total_user_distance)
        user_dtw_list.append(user[i][0]-user_start_elev)
        #plt.scatter(total_user_distance, user[i][0])
        #print(arr[0][i][0])
    for j in range (len(segment)):
         total_segment_distance1 += segment[j][1]
         seg_dtw_distance.append(total_segment_distance1)
         segment_dtw_list.append(segment[j][0]-segment_start_elev)
         #plt.scatter(total_segment_distance1, segment[j][0])
    #plt.draw()
    return ((user_dtw_list,user_dtw_distance), (segment_dtw_list,seg_dtw_distance))

In [50]:
from scipy.interpolate import interp1d
import numpy as np
# this is to correct the length of the segment for use in better DTW algorithms, like 
# the ones I have imported. My implementation does mot require this. 
def correct_segment(l):
    #l = graphing(arr[0], claremont_turkey_trot) #(elev, distance), (elev, distance)
    #print(l[0][1])
    #plt.plot(l[0][1],l[0][0],'o')
    f = interp1d(l[1][1],l[1][0], kind='cubic')
    segment_corrected_length = []
    desired_length = len(l[0][0])
    divided = l[1][1][-1]//desired_length
    for i in range(desired_length-1):
        segment_corrected_length.append(f(i*divided))
    #plt.plot(l[1][1], f(l[1][1]), '-')
    #plt.show()
    return segment_corrected_length


In [51]:
from scipy.interpolate import interp1d
import numpy as np
#!pip install dtaidistance
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis
#l = graphing(arr[0], claremont_turkey_trot)

# this does the actual comparison and exports tuples of (DTW distance, user times)
def DTW_comparison(user, segment):
    distances = []
    user_times = []
    d_t = []
    for pot in range(len(user)-1):
        t1 = user[pot][0][1]
        t2 = user[pot][-1][1]
        #print(t1)
        user_times.append((t2-t1).total_seconds())
        l = graphing(user[pot], segment) #(elev, distance), (elev, distance)
        first = l[0][0]
        second = l[1][0]
        #f2 = interp1d(l[0][0], l[0][1], kind='cubic')
        tempDTW = dtw(first, second)
        distances.append(tempDTW[-1][-1])
        #distances.append(dtw.distance(first, second))

    
    for d in range(len(distances)):
        d_t.append((distances[d],user_times[d]))
    return d_t
    


In [53]:
# code to use imported DTW visualization

#!pip install dtaidistance
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis
#s = correct_segment(l)
#path = dtw.warping_path(l[0][0], s)
#dtwvis.plot_warping(l[0][0], s, path, filename="warp.png")
#distance = dtw.distance(l[0][0], s)

In [54]:
conv = Converter() 
# method to batch convert .FIT to .GPX 
# UPDATE: THIS DOES NOT WORK WITH ELEVATION DATA
def fit_to_gpx(fileDirectory):
    conv.fit_to_gpx_bulk(dir_in=fileDirectory, dir_out='data/User')



In [119]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

def get_segment_distance_time_grade(segment_id):
    driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')
    url = f"https://www.strava.com/segments/{segment_id}?filter=overall"
    driver.get(url)
    #login_to_strava()
    distance_element = driver.find_element(By.XPATH,'//*[@id="segment"]/div/div[3]/ul/li[1]/div/b')
    grade_element = driver.find_element(By.XPATH,'//*[@id="segment"]/div/div[3]/ul/li[3]/div/b')
    try:
        # check if there is a VAM
         leader_time_element = driver.find_element(By.XPATH,'//*[@id="segment-leaderboard"]/div/table/tbody/tr[1]/td[5]')
    except Exception:
         leader_time_element = driver.find_element(By.XPATH,'//*[@id="segment-leaderboard"]/div/table/tbody/tr[1]/td[4]')
    
    return distance_element.text,leader_time_element.text,grade_element.text

In [142]:
import time
def data_to_display(act, segment_best_time):
    
    count = 0
    total_time = 0
    min_time = 1000000000
    min_tuple = None
    
    for i in act:
        if i[0] < 3000:
            total_time += i[1]
            count+=1
            if i[1] < min_time:
                min_time = i[1]
                min_tuple = i
    if count != 0:
        average_time = total_time/count   
        average_time_res = time.gmtime(average_time)
        ave_res = time.strftime("%H:%M:%S",average_time_res)
        min_time_res = time.gmtime(min_time)
        min_res = time.strftime("%H:%M:%S",min_time_res)
    else: 
        ave_res =  "not enough data"
        min_res =  "not enough data"
        min_tuple= ("none", "not enough data")
    print(f"The current segment leader ran the segment in {segment_best_time[1]}")
    print(f"Based on your prior performance, the average time you could run this segment in is {ave_res}.")
    print(f"The fastest time you could run this segment is {min_res} based on a run that is {min_tuple[0]} similar to the segment based on DTW.")



In [120]:
import os
# this is the main method, runs it through and returns list of (DTW distance, user times)
def compare_user_activities_to_segment(segmentPath, segmentID):
    # scrape for segment
    data = []
    seg = read_segment(segmentPath)
    #print(seg)
    # go through user's activities
    directory = os.fsencode('/Users/wjones/School/DSA/HunterXKOM/data/Mix_data')    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".gpx"): 
            arrOne = file_to_elevation_arr('/Users/wjones/School/DSA/HunterXKOM/data/Mix_data/'+filename)
            #print(arrOne)
            arr = sweep_user_activity(arrOne, seg, 5)
            data += DTW_comparison(arr, seg)
        else:
            continue
    return data_to_display(data, get_segment_distance_time_grade(segmentID))

## Testing 
- Took 13 activities from James's strava representing workouts and hard hill runs. 
- Nest, we compare those activities to segments James already is the leader on, stored in data/Mix_data
- Results are below. For most segments this algorithm worked, but for a few the signficant elevation meant we don't have an accurate result.

In [135]:
compare_user_activities_to_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/5336817.json', "5336817")


/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_47319/2394875006.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')


The current segment leader ran the segment in 2:29
Based on your prior performance, the average time you could run this segment in is 00:03:31.
The fastest time you could run this segment is 00:02:05 based on a run that is 113.06000000000518 similar to the segment based on DTW.


In [138]:
compare_user_activities_to_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/8224728.json', "8224728")


/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_47319/2394875006.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')


The current segment leader ran the segment in 8:01
Based on your prior performance, the average time you could run this segment in is 00:11:33.
The fastest time you could run this segment is 00:11:30 based on a run that is 156.7700000000001 similar to the segment based on DTW.


In [141]:
compare_user_activities_to_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/8224741.json', "8224741")

/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_47319/2394875006.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')


The current segment leader ran the segment in 10:01
Based on your prior performance, the average time you could run this segment in is 00:16:19.
The fastest time you could run this segment is 00:15:45 based on a run that is 1669.3199999999824 similar to the segment based on DTW.


In [144]:
compare_user_activities_to_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/4626685.json', "4626685")

/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_47319/2394875006.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')


The current segment leader ran the segment in 2:07
Based on your prior performance, the average time you could run this segment in is 00:22:06.
The fastest time you could run this segment is 00:17:27 based on a run that is 1328.0899999999992 similar to the segment based on DTW.


Note: Although the algorithm does not say James can get his own segments above, this makes sense, as we did not feed it any run user data with this high of an effort at such a grade.  These segments are from the same steep, fast run.

In [145]:
compare_user_activities_to_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/20223561.json', "20223561")


/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_47319/2394875006.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')


The current segment leader ran the segment in 10:21
Based on your prior performance, the average time you could run this segment in is 00:14:43.
The fastest time you could run this segment is 00:10:03 based on a run that is 347.7299999999986 similar to the segment based on DTW.


In [146]:
compare_user_activities_to_segment('/Users/wjones/School/DSA/HunterXKOM/data/Elevation_JSON/1410421.json', "1410421")

/var/folders/01/r024mm7x74qd70l_5q2qs0hm0000gn/T/ipykernel_47319/2394875006.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/James/Downloads/chromedriver')


The current segment leader ran the segment in 13:54
Based on your prior performance, the average time you could run this segment in is 00:29:38.
The fastest time you could run this segment is 00:25:46 based on a run that is 1802.1899999999941 similar to the segment based on DTW.
